## all functions

In [ ]:
import os
import numpy as np
from xml.dom.minidom import parse
from shapely.geometry import asPolygon
import glob
import shutil
import zipfile


# ------------------------------classify by county------------------------------
def kml2polygon(kml_file):
    # unzip kmz to get kml
    if kml_file.endswith('.kmz'):
        dir_name = os.path.dirname(kml_file)
        with zipfile.ZipFile(kml_file, 'r') as f:
            files = f.namelist()
            f.extract(files[0], dir_name)
        kml_file = os.path.join(dir_name, 'doc.kml')

    domTree = parse(kml_file)
    rootNode = domTree.documentElement
    Placemarks = rootNode.getElementsByTagName('Placemark')

    polygon_dict = {}
    j = 0

    for Placemark in Placemarks:
        name = Placemark.getElementsByTagName('name')[0].childNodes[0].data
        ploygon = Placemark.getElementsByTagName('Polygon')[0]
        outerBoundaryIs = ploygon.getElementsByTagName('outerBoundaryIs')[0]
        LinearRing = outerBoundaryIs.getElementsByTagName('LinearRing')[0]
        coordinates = LinearRing.getElementsByTagName(
            'coordinates')[0].childNodes[0].data
        lon_lat = [i.split(',')[0:2] for i in coordinates.strip().split(' ')]
        polygon_dict[name] = np.asarray(lon_lat, dtype='float64')
        j += 1
    return polygon_dict


def intersect(point, s_point, e_point):
    if s_point[1] == e_point[
            1]:  # parallel and coincident with the ray，s_point coincides with s_point
        return False
    if s_point[1] > point[1] and e_point[1] > point[
            1]:  # line segment is above the ray
        return False
    if s_point[1] < point[1] and e_point[1] < point[
            1]:  # line segment under the ray
        return False
    if s_point[1] == point[
            1] and e_point[1] > point[1]:  # point coincides with s_point
        return False
    if e_point[1] == point[
            1] and s_point[1] > point[1]:  # point coincides with e_point
        return False
    # line segment is to the left of the ray
    if s_point[0] < point[0] and e_point[0] < point[0]:
        return False

    xseg = e_point[0]-(e_point[0]-s_point[0])*(e_point[1]-point[1]) / \
        (e_point[1]-s_point[1])  # find the intersection
    if xseg < point[0]:  # intersection is to the left of point
        return False
    return True


def inpolygon(point, polygon):
    num = 0  # number of intersection
    for i in range(len(polygon) - 1):
        if intersect(point, polygon[i], polygon[i + 1]):
            num += 1
    return True if num % 2 == 1 else False


def get_centroid(txt):
    data = np.loadtxt(txt)
    lon_lat = data[1:, 1:3]
    polygon = asPolygon(lon_lat)
    return np.asarray(polygon.centroid)


def get_center(txt):
    data = np.loadtxt(txt)
    lon = data[1:, 1]
    lat = data[1:, 2]
    lon_mean = np.mean(lon)
    lat_mean = np.mean(lat)
    return [lon_mean, lat_mean]


# ------------------------------classify by county------------------------------


# ------------------------------classify by velocity------------------------------
def load_data(ts_file):
    try:
        data = np.loadtxt(ts_file)
        vel = data[1:, 3]
        return vel
    except:
        return [0]


def judge_class(vel, filter):
    min_vel = np.min(vel)
    max_vel = np.max(vel)
    positive_num = np.where(vel >= 0)[0].size
    negative_num = np.where(vel <= 0)[0].size
    positive_rate = positive_num / vel.size
    negative_rate = negative_num / vel.size
    if positive_rate > negative_rate:
        for i in range(len(filter) - 1):
            min_f = filter[i]
            max_f = filter[i + 1]
            if max_vel >= min_f and max_vel < max_f:
                return str(max_f)
            elif max_vel < filter[0]:
                return str(filter[0])
            elif max_vel > filter[-1]:
                return str(filter[-1])
    else:
        for i in range(len(filter) - 1):
            min_f = filter[i]
            max_f = filter[i + 1]
            if min_vel >= min_f and min_vel < max_f:
                return str(min_f)
            elif min_vel < filter[0]:
                return str(filter[0])
            elif min_vel > filter[-1]:
                return str(filter[-1])


# ------------------------------classify by velocity------------------------------

## classify by county

In [ ]:
county = kml2polygon('/media/ly/文件/sc_prj/Sentinel-1/按县分级/所有县.kmz')
txts = glob.glob(
    os.path.join('/media/ly/文件/sc_prj/Sentinel-1/按县分级/有序号', '*.txt'))

save_path = '/media/ly/文件/sc_prj/Sentinel-1/按县分级/按县分类'

kmz_dir = '/media/ly/文件/sc_prj/Sentinel-1/按县分级/升降轨合并_all'

for txt in txts:
    for key, value in county.items():
        # create dir
        dir_name = os.path.join(save_path, key)
        if not os.path.isdir(dir_name):
            os.mkdir(dir_name)
        # get centroid
        # centroid = get_centroid(txt)
        centroid = get_center(txt)
        # judet
        if inpolygon(centroid, value):
            name = os.path.basename(txt)[:-4]
            kmz = os.path.join(kmz_dir, name + '.kmz')
            if os.path.isfile(kmz):
                shutil.move(kmz, dir_name)

## classify by track direction

In [ ]:
path = '/media/ly/文件/sc_prj/Sentinel-1/按县分级/按县分类'

kmzs = glob.glob(path + '/*/*.kmz')

des = list(range(1, 441)) + list(range(813, 879))

asc = list(range(441, 813))

asc_num = 0
des_num = 0
asc_des_num = 0


for kmz in kmzs:
    dir_name = os.path.dirname(kmz)

    asc_dir = os.path.join(dir_name, '升轨')
    if not os.path.isdir(asc_dir):
        os.mkdir(asc_dir)
    des_dir = os.path.join(dir_name, '降轨')
    if not os.path.isdir(des_dir):
        os.mkdir(des_dir)
    asc_des_dir = os.path.join(dir_name, '升降轨')
    if not os.path.isdir(asc_des_dir):
        os.mkdir(asc_des_dir)

    colorbar_num = 0
    with zipfile.ZipFile(kmz, 'r') as f:
        files = f.namelist()
        for file in files:
            if 'colorbar' in file:
                colorbar_num += 1

    if colorbar_num >= 2:
        shutil.move(kmz, asc_des_dir)
        asc_des_num += 1
    else:
        name = int(os.path.basename(kmz)[:-4])
        if name in des:
            shutil.move(kmz, des_dir)
            des_num += 1
        if name in asc:
            shutil.move(kmz, asc_dir)
            asc_num += 1
            
print('asc:', asc_num)
print('des:', des_num)
print('asc_des', asc_des_num)

## classify by velocity

### rename

In [ ]:
dir_name = '/media/ly/文件/sc_prj/Sentinel-1/result_last/cut_res/cut-221'

files = glob.glob(dir_name + '/*.txt')
for f in files:
    new_name = str(files.index(f) + 887) + '.txt'
    path = os.path.join(dir_name, new_name)

    os.rename(f, path)

### classify and make kmz

In [ ]:
def make_kmz_all(ts_dir, filter, kmz_scripts):
    files = glob.glob(os.path.join(ts_dir, '*.txt'))
    for f in files:
        vel = load_data(f)
        if len(vel) > 1:
            name = judge_class(vel, filter)
            kmz_script = os.path.join(kmz_scripts,
                                      f"make_kmz_timeseries{name}.py")
            js_sctipt = os.path.join(kmz_scripts, 'dygraph-combined.js')
            out_kmz = os.path.join(ts_dir, os.path.basename(f)[:-4])
            print(f"\rprocessing {files.index(f) + 1}/{len(files)}",
                  end=" ",
                  flush=True)
            cmd_str = f"python3 '{kmz_script}' -t '{f}' -j '{js_sctipt}' -o '{out_kmz}' -s 0.5"
            os.system(cmd_str)


filter = sorted(
    [-60, -50, -40, -30, -20, -15, -10, 0, 10, 15, 20, 30, 40, 50, 60])
ts_dir = '/media/ly/文件/sc_prj/Sentinel-1/result_last/cut_res/cut22-23'
kmz_scripts = '/media/ly/文件/sc_prj/Sentinel-1/result_last/cut_res/make_kmz'

make_kmz_all(ts_dir, filter, kmz_scripts)